In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")
from syft import autocache

✅ The installed version of syft==0.8.0.post2 matches the requirement >=0.8 and the requirement <0.8.1


In [ ]:
import pandas as pd
raw_data = pd.read_pickle("../datasets/sample.pkl")

In [14]:
node = sy.orchestra.launch(name="test-domain_api", port="auto", dev_mode=True, reset=True)


Starting test-domain_api server on 0.0.0.0:8421

SQLite Store Path:
!open file:///tmp/169ee8d9e984406ebd1c7698e66a923f.sqlite

Waiting for server to start...

INFO:     Started server process [4279]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8421 (Press CTRL+C to quit)


INFO:     127.0.0.1:54950 - "GET /api/v2/metadata HTTP/1.1" 200 OK
. Done.
INFO:     127.0.0.1:54954 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:54954 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:54954 - "GET /api/v2/api?verify_key=855efeabbf524904b1902aeb0bb7bcf64bb441dccc0e51cbdacc65d2d639769c HTTP/1.1" 200 OK


In [15]:
domain_client = node.login(email="info@openmined.org", password="changethis")


In [16]:
domain_client

/home/hossam/anaconda3/envs/new_version/lib/python3.9/site-packages/IPython/core/formatters.py:367: FormatterWarning: text/html formatter returned invalid type <class 'type'> (expected <class 'str'>) for object: <SyftClient - test-domain_api <169ee8d9e984406ebd1c7698e66a923f>: HTTPConnection: http://localhost:8421>
  warnings.warn(


<SyftClient - test-domain_api <169ee8d9e984406ebd1c7698e66a923f>: HTTPConnection: http://localhost:8421>

In [19]:
domain_client.privacy_budget

AttributeError: SyftClient object has no attribute privacy_budget.

In [ ]:
data_subjects = domain_client.api.services.data_subject.get_all()


In [ ]:
data_subjects

In [ ]:
assert len(data_subjects) == 0


In [ ]:
import glob
import pandas as pd
dfs = []
filepaths = glob.glob('../datasets/EMAKI_preprocessed/mouse/*.csv')
assert len(filepaths) > 0, 'Dataset not found! Have you already been converting it?'
for filepath in filepaths:
     filename = filepath.split('\\')[-1].split('/')[-1].split('.')[0]
     dfs.append(pd.read_csv(filepath))

In [ ]:
user = sy.DataSubject(name="user", aliases=["user_code"])
dict = {}
for i in range (39):
   user.add_member(sy.DataSubject(name=f"user {i}", aliases=[f"user_code {i}"]))

In [ ]:
registry = domain_client.data_subject_registry
response = registry.add_data_subject(user)

In [ ]:
domain_client.data_subject_registry


In [ ]:
data_subjects = domain_client.api.services.data_subject.get_all()


In [ ]:
for data_subject in data_subjects:
    print (data_subject)

In [ ]:
mouse_data = raw_data[raw_data['type'].isin(['mousemove', 'mousedown', 'mouseup'])]
mouse_data=mouse_data.drop(['ID', 'INVALID', 'type', 'value', 'mu'], axis=1)

keyboard_data = raw_data[raw_data['type'].isin(['keyup', 'keydown'])]
keyboard_data=keyboard_data.drop(['ID', 'INVALID', 'X', 'Y', 'resolutionX', 'resolutionY', 'mu'], axis=1)
keystrokes = keyboard_data['value'].unique()
# encode the keystrokes
# 0 > keydown 1 > keyup
# 0 > 112 keystorkes
for i in range(len(keystrokes)):
    keyboard_data['value'].replace(keystrokes[i], i, inplace=True)



In [ ]:
asset = mouse_data[0:1000]
mock_data = mouse_data[1000:2000]


In [ ]:
asset2 = keyboard_data[0:1000]
mock_data2 = keyboard_data[1000:2000]

In [ ]:
dataset = sy.Dataset(name="mouse data")
dataset.set_description("""mouse data """)

In [ ]:
dataset.add_citation("Person, place or thing")
dataset.add_url("https://github.com/OpenMined/datasets/tree/main/trade_flow")

In [ ]:
dataset.add_contributor(role=sy.roles.UPLOADER, 
                                name="Hossam Elfar ", 
                                email="hossam@openmined.org",
                                note="Andrew runs this domain and prepared the dataset metadata.")

dataset.add_contributor(role=sy.roles.EDITOR, 
                                name="hossam elfar", 
                                email="madhava@openmined.org",
                                note="Madhava tweaked the description to add the URL because Andrew forgot.")

In [ ]:
dataset.contributors


In [ ]:
ctf = sy.Asset(name="mouse_data")
ctf.set_description("""all the datas""")

In [ ]:
ctf.add_contributor(role=sy.roles.UPLOADER, 
                      name="hossam elfar", 
                      email="andrew@openmined.org",
                      note="Andrew runs this domain and prepared the asset.")

In [ ]:
ctf.set_obj(asset)
ctf.set_shape((1000, 13))
ctf.add_data_subject(data_subjects[9])

In [ ]:
ctf.set_mock(mock_data, mock_is_real=True)


In [ ]:
dataset.add_asset(ctf)


In [ ]:
dataset.remove_asset(name=ctf.name)


In [ ]:
dataset

In [ ]:
domain_client.upload_dataset(dataset)


In [ ]:
dataset2 = sy.Dataset(name="keyboard data")
dataset2.set_description("""keyboard data """)

In [ ]:
dataset2.add_citation("Person, place or thing")
dataset2.add_url("https://github.com/OpenMined/datasets/tree/main/trade_flow")

In [ ]:
dataset2.add_contributor(role=sy.roles.UPLOADER, 
                                name="Hossam Elfar ", 
                                email="andrew@openmined.org",
                                note="Andrew runs this domain and prepared the dataset metadata.")

dataset2.add_contributor(role=sy.roles.EDITOR, 
                                name="Madhava Jay", 
                                email="madhava@openmined.org",
                                note="Madhava tweaked the description to add the URL because Andrew forgot.")

In [ ]:
ctf = sy.Asset(name="keyboard data")
ctf.set_description("""all the datas""")

In [ ]:
ctf.set_obj(asset2)
ctf.set_shape((1000, 13))
ctf.add_data_subject(data_subjects[9])

In [ ]:
ctf.set_mock(mock_data2, mock_is_real=True)


In [ ]:
dataset2.add_asset(ctf)

In [ ]:
ctf.add_contributor(role=sy.roles.UPLOADER, 
                      name="Hossam ELFAR", 
                      email="andrew@openmined.org",
                      note="Andrew runs this domain and prepared the asset.")

In [ ]:
domain_client.upload_dataset(dataset2)


In [ ]:
datasets = domain_client.api.services.dataset.get_all()


In [ ]:
datasets

In [ ]:
real = domain_client.datasets[0].assets[0].data


In [ ]:
real

In [ ]:
assert asset.equals(real.syft_action_data)


In [ ]:
################################################## here we gonna take client expereince

In [ ]:
import syft as sy
sy.requires(">=0.8,<0.8.1")
from syft.client.api import NodeView

In [ ]:
guest_domain_client = node.client


In [ ]:
guest_credentials = guest_domain_client.credentials
guest_credentials

In [ ]:
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")


In [ ]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")


In [ ]:
datasets =guest_domain_client.datasets


In [ ]:
datasets

In [ ]:
mouse_dataset = datasets[0]


In [ ]:
mouse_dataset

In [ ]:
assert guest_domain_client.credentials != guest_credentials


In [ ]:
mock = mouse_dataset.assets[0].mock
mock

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(mouse_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def average_x_coordinatees(mouse_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = mouse_data
    total = df['X'].mean()*df['resolutionX'].mean()
    return (float(total)+float(noise))


In [ ]:
sum_x_values(mouse_data=mock)

In [ ]:
node_view = NodeView.from_api(guest_domain_client.api)
assert node_view in sum_x_values.kwargs

In [ ]:
assert "mouse_data" in sum_x_values.kwargs[node_view]


In [ ]:
assert sum_x_values.input_policy_init_kwargs[node_view]["mouse_data"] == mock.id


In [ ]:
sum_x_values

In [ ]:
sum_x_values.code

In [ ]:
submitted_req = guest_domain_client.api.services.code.request_code_execution(sum_x_values)


In [ ]:
assert len(guest_domain_client.api.services.code.get_all()) == 1


In [ ]:
project_create = sy.Project(
    name="My Cool UN Project",
    description="""Hi, I want to calculate the sum volume of x with my cool code.""",
    shareholders=[guest_domain_client],
    user_email_address = "jane@caltech.edu",
    users = [guest_domain_client]
)
project_create

In [ ]:
projects = project_create.start()

In [ ]:
projects

In [ ]:
project = projects[0]

In [ ]:
project

In [ ]:
project.add_request(request=submitted_req)


In [ ]:
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)

In [ ]:
result = guest_domain_client.api.services.code.sum_x_values(mouse_data=mock.id)
result

In [ ]:
assert isinstance(result, sy.SyftNotReady)


In [ ]:
######################### review code 


In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")


In [ ]:
domain_client.notifications


In [ ]:
messages = domain_client.notifications.get_all_unread()
messages

In [ ]:
read_messages = domain_client.notifications.get_all_read()
read_messages

In [ ]:
project_message = None
for message in messages:
    if issubclass(message.linked_obj.object_type, sy.service.project.project.Project):
        project_message = message

In [ ]:
project = project_message.link
project

In [ ]:
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)


In [ ]:
request = project.events[0].request
request

In [ ]:
func = request.changes[0].link
func

In [ ]:
print(func.code)


In [ ]:
func.node_uid


In [ ]:
func.assets


In [ ]:
asset = func.assets[0]
asset

In [ ]:
mock = asset.mock
mock

In [ ]:
data = asset.data
data

In [ ]:
op = func.output_policy_type
op

In [ ]:
print(op.policy_code)


In [ ]:
print(func.raw_code)


In [ ]:
users_function = func.unsafe_function


In [ ]:
real_result = users_function(mouse_data=data)
real_result

In [ ]:
result = request.accept_by_depositing_result(real_result)
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)


In [ ]:
########################### client download results

In [ ]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")


In [ ]:
guest_domain_client.datasets

In [ ]:
asset = domain_client.datasets[0].assets[0].data
asset

In [ ]:
real_result = domain_client.api.services.code.sum_x_values(mouse_data=asset)
real_result

In [ ]:
ops = domain_client.api.services.code[-1].output_policy
ops